In [3]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from skimage.transform import resize

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
NUM_CLASSES = 10
BATCH_SIZE = 5
NUM_EPOCHS = 10
use_data_aug = False

In [5]:
from tqdm import tqdm
# img_arr is of shape (n, h, w, c)
def resize_image_arr(img_arr):
    x_resized_list = []
    for i in tqdm(range(img_arr.shape[0])):
        img = img_arr[0]
        resized_img = resize(img, (224, 224))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)




In [17]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
import pandas as pd

x_df= x_test.reshape(10000, x_test.shape[1] * x_test.shape[2] *x_test.shape[3] )

imgs = pd.DataFrame() 
df_labels = pd.DataFrame() 

imgs = imgs.append(pd.DataFrame(x_df)) 
df_labels = df_labels.append(pd.DataFrame(y_test)) 
imgs['labels'] = df_labels 

imgs

In [7]:
x_test = resize_image_arr(x_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:35<00:00, 104.19it/s]


In [19]:
x_test = x_test/255.0
np.min(x_test),np.max(x_test)

(0.0, 1.0)

In [20]:
import pandas as pd
x_test_flat = x_test.reshape(-1,3072)
feat_cols = ['pixel'+str(i) for i in range(x_test_flat.shape[1])]
df_cifar = pd.DataFrame(x_test_flat,columns=feat_cols)
df_cifar['label'] = y_test
print('Size of the dataframe: {}'.format(df_cifar.shape))

Size of the dataframe: (10000, 3073)


In [22]:
# Import PCA from sklearn
from sklearn.decomposition import PCA

pca_cifar = PCA(n_components=2)
principalComponents_cifar = pca_cifar.fit_transform(df_cifar.iloc[:,:-1])
principal_cifar_Df = pd.DataFrame(data = principalComponents_cifar
             , columns = ['principal component 1', 'principal component 2'])
principal_cifar_Df['y'] = y_test
principal_cifar_Df.head()

,principal component 1,principal component 2,y
0,-3.694406,0.871710,3
1,9.688801,-9.689979,8
2,4.423473,-8.887094,8
3,7.755905,-3.889165,0
4,-5.463532,4.298671,6


In [8]:
# Convert class vectors to binary class matrices.
#y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Normalize the data
x_test = x_test.astype('float32')
x_test /= 255

In [ ]:
base_model = ResNet50(include_top=False, weights= None)

In [ ]:
base_model.summary()

In [ ]:
img_idx = imgs.loc[imgs['labels'] == i][:100]

abc = img_idx.iloc[:, :3072]

for idx, row in abc.iterrows():
    print(idx)

In [ ]:
x_test.shape

In [ ]:
from sklearn.preprocessing import normalize, scale, MinMaxScaler

(x_train_SAMP, y_train_SAMP), (x_test_SAMP, y_test_SAMP) = cifar10.load_data()

x_train_SAMP =[]
y_train_SAMP =[]

result = {}
LAYER_COUNTER = 0
for idx in range(len(base_model.layers)):

    
    if base_model.get_layer(index = idx).__class__.__name__ == 'Conv2D':
        # PREPARE FOR JSON 
        result['dataset_'+str(LAYER_COUNTER)]= []
    
        for i in range(0,10):
            
            
            img_idx = imgs.loc[imgs['labels'] == i][:100]
            
            temp_img_arr = []
            for index, row in img_idx.iloc[:, :3072].iterrows():
                #print(x_test_SAMP[index].shape)
                #Xtest = np.reshape(data[b"data"][3], (-1, 3, 32, 32))
                #Xtest = np.transpose(Xtest, (0,2,3,1))

                #x1 = np.reshape(np.asarray(row),(-1,32,32,3))
                #x1 = np.transpose(x1, (0,2,3,1))
                temp_img_arr.append(x_test[index])
            
            #print(temp_img_arr)
            #temp_img_arr = np.array(temp_img_arr).reshape(-1,32,32,3)
            temp_img_arr = np.stack(temp_img_arr,axis=0)
            print(temp_img_arr.shape)
            
            x_batch = resize_image_arr(temp_img_arr)
            print(x_batch.shape)

            x_batch = x_batch.astype('float32')
            x_batch /= 255
            
            
            layerName = base_model.get_layer(index = idx).name
            print(idx, layerName)

            model_cut = Model(inputs=base_model.inputs, output=base_model.layers[idx].output)
            preds = model_cut.predict(x_batch)
            print('preds',preds.shape)

            preds_flat = preds.reshape(preds.shape[0],preds.shape[1] * preds.shape[2] * preds.shape[3])
            print(preds_flat.shape)

            import pandas as pd 

            X = preds_flat
            # Standardize the features
            #X = StandardScaler().fit_transform(preds_flat)
            #X = normalize(preds_flat)
            #X = scale(preds_flat)

            #mm_scaler = MinMaxScaler()
            #X = mm_scaler.fit_transform(preds_flat)
            #mm_scaler.transform(X)

            # Preview X
            pd.DataFrame(data=X).head()

            # Import PCA from sklearn
            from sklearn.decomposition import PCA

            # Instantiate PCA
            pca = PCA(n_components=2)

            # Fit PCA to features
            principalComponents = pca.fit_transform(X)
            
            CLASS_PCA_RES = []
            for ipc in range(principalComponents.shape[0]):
                arr_classPt = {
                    "uid" : "",
                    "filename" : "",
                    "comp1": str(principalComponents[ipc][0]),
                    "comp2": str(principalComponents[ipc][1])
                }
                CLASS_PCA_RES.append(arr_classPt)
                
            result["dataset_"+str(LAYER_COUNTER)].append(CLASS_PCA_RES)

            # Create a new dataset from principal components 
            df = pd.DataFrame(data = principalComponents, 
                              columns = ['PC1', 'PC2'])

            target = pd.Series(y_test_SAMP[:100].reshape(100,), name='target')

            result_df = pd.concat([df, target], axis=1)
            print(result_df)
        
        LAYER_COUNTER+= 1
    ##if idx > 10:
     #   break


In [ ]:
x_test[:10].shape

In [ ]:
from sklearn.preprocessing import normalize, scale, MinMaxScaler

(x_train_SAMP, y_train_SAMP), (x_test_SAMP, y_test_SAMP) = cifar10.load_data()

x_train_SAMP =[]
y_train_SAMP =[]



for idx in range(len(base_model.layers)):
    if base_model.get_layer(index = idx).__class__.__name__ == 'Conv2D':
        for i in range(0,10):
            x_batch = x_test[:100]
            
           
            
            layerName = base_model.get_layer(index = idx).name
            print(idx, layerName)

            model_cut = Model(inputs=base_model.inputs, output=base_model.layers[idx].output)
            preds = model_cut.predict(x_batch)
            print('preds',preds.shape)

            preds_flat = preds.reshape(preds.shape[0],preds.shape[1] * preds.shape[2] * preds.shape[3])
            print(preds_flat.shape)

            import pandas as pd 

            X = preds_flat
            # Standardize the features
            #X = StandardScaler().fit_transform(preds_flat)
            #X = normalize(preds_flat)
            #X = scale(preds_flat)

            #mm_scaler = MinMaxScaler()
            #X = mm_scaler.fit_transform(preds_flat)
            #mm_scaler.transform(X)

            # Preview X
            pd.DataFrame(data=X).head()

            # Import PCA from sklearn
            from sklearn.decomposition import PCA

            # Instantiate PCA
            pca = PCA(n_components=2)

            # Fit PCA to features
            principalComponents = pca.fit_transform(X)

            # Create a new dataset from principal components 
            df = pd.DataFrame(data = principalComponents, 
                              columns = ['PC1', 'PC2'])

            target = pd.Series(y_test_SAMP[:10].reshape(10,), name='target')

            result_df = pd.concat([df, target], axis=1)
            print(result_df)
        
    if idx > 10:
        break


In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd 

from sklearn.preprocessing import normalize
# Standardize the features
#X = StandardScaler().fit_transform(preds_flat)

# Preview X
pd.DataFrame(data=preds_flat).head()

# Import PCA from sklearn
from sklearn.decomposition import PCA

# Instantiate PCA
pca = PCA(n_components=2)

# Fit PCA to features
principalComponents = pca.fit_transform(preds_flat)

In [ ]:
# Visualize Principal Components with a scatter plot
import matplotlib.pyplot as plt 

fig = plt.figure(figsize = (12,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('First Principal Component ', fontsize = 15)
ax.set_ylabel('Second Principal Component ', fontsize = 15)
ax.set_title('Principal Component Analysis (2PCs) for Iris Dataset', fontsize = 20)

targets = [3, 1, 8, 0 ,6]
colors = ['r', 'g', 'b', 'c', 'y']
for target, color in zip(targets, colors):
    indicesToKeep = y_test_SAMP[:10].reshape(10,) == target
    ax.scatter(result_df.loc[indicesToKeep, 'PC1'], 
               result_df.loc[indicesToKeep, 'PC2'], 
               c = color, 
               s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
result

In [ ]:
import json 

result["n_layers"] = str(LAYER_COUNTER)

with open('sample_json3.json', 'w') as outfile:
    json.dump(result, outfile)

In [ ]:
np.min(x_test),np.max(x_test)